<a href="https://colab.research.google.com/github/MadmanMarble/Planet_U-NET/blob/main/Planet_training_data_organization_032024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 57.1 MB/s eta 0:00:00


In [3]:
import rasterio
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import shutil
import hashlib

In [4]:
!mkdir /content/OneYear
!mkdir /content/OneYear/utm4
!mkdir /content/OneYear/utm4/images
!mkdir /content/OneYear/utm4/labels

!mkdir /content/OneYear/utm5
!mkdir /content/OneYear/utm5/images
!mkdir /content/OneYear/utm5/labels

!mkdir /content/TwoYears
!mkdir /content/TwoYears/utm4
!mkdir /content/TwoYears/utm4/images
!mkdir /content/TwoYears/utm4/labels

!mkdir /content/TwoYears/utm5
!mkdir /content/TwoYears/utm5/images
!mkdir /content/TwoYears/utm5/labels


# OneYear UTM 4 Data

In [5]:
!unzip /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4_updatedExports_march20/utm4_OneYear_stacked.zip -d /content/OneYear/utm4/images/

Archive:  /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4_updatedExports_march20/utm4_OneYear_stacked.zip
   creating: /content/OneYear/utm4/images/content/folder_stack/OneYear/
  inflating: /content/OneYear/utm4/images/content/folder_stack/OneYear/735_stacked.tif  
  inflating: /content/OneYear/utm4/images/content/folder_stack/OneYear/1009_stacked.tif  
  inflating: /content/OneYear/utm4/images/content/folder_stack/OneYear/1347_stacked.tif  
  inflating: /content/OneYear/utm4/images/content/folder_stack/OneYear/2342_stacked.tif  
  inflating: /content/OneYear/utm4/images/content/folder_stack/OneYear/1711_stacked.tif  
  inflating: /content/OneYear/utm4/images/content/folder_stack/OneYear/967_stacked.tif  
  inflating: /content/OneYear/utm4/images/content/folder_stack/OneYear/30_stacked.tif  
  inflating: /content/OneYear/utm4/images/content/folder_stack/OneYear/1276_stacked.tif  
  inflating: /content/OneYear/utm4/images/content/folder_stack/OneYear/103

In [7]:
# Define the source and destination directories
source_dir = "/content/OneYear/utm4/images/content/folder_stack/OneYear"
destination_dir = "/content/OneYear/utm4/images"

# List all .tif files in the source directory
tif_files = [file for file in os.listdir(source_dir) if file.endswith(".tif")]

# Move each .tif file to the destination directory
for file in tif_files:
    # Define the source and destination file paths
    source_file = os.path.join(source_dir, file)
    destination_file = os.path.join(destination_dir, file)

    # Move the file to the destination directory
    shutil.move(source_file, destination_file)


In [8]:
# Define the folder containing .tif files
images_folder = '/content/OneYear/utm4/images'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')


The number of .tif files in the folder is: 2629


In [26]:
!unzip /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4_updatedExports_march20/utm4_2018_nwi_ccap_filled_07.zip -d /content/OneYear/utm4/labels/

Archive:  /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4_updatedExports_march20/utm4_2018_nwi_ccap_filled_07.zip
   creating: /content/OneYear/utm4/labels/content/exports/nwi_ccap_filled/
  inflating: /content/OneYear/utm4/labels/content/exports/nwi_ccap_filled/1818_nwi_ccap_filled.tif  
  inflating: /content/OneYear/utm4/labels/content/exports/nwi_ccap_filled/1250_nwi_ccap_filled.tif  
  inflating: /content/OneYear/utm4/labels/content/exports/nwi_ccap_filled/1832_nwi_ccap_filled.tif  
  inflating: /content/OneYear/utm4/labels/content/exports/nwi_ccap_filled/2389_nwi_ccap_filled.tif  
  inflating: /content/OneYear/utm4/labels/content/exports/nwi_ccap_filled/1660_nwi_ccap_filled.tif  
  inflating: /content/OneYear/utm4/labels/content/exports/nwi_ccap_filled/1471_nwi_ccap_filled.tif  
  inflating: /content/OneYear/utm4/labels/content/exports/nwi_ccap_filled/1201_nwi_ccap_filled.tif  
  inflating: /content/OneYear/utm4/labels/content/exports/nwi_ccap_fille

In [27]:
# Define the source and destination directories
source_dir = "/content/OneYear/utm4/labels/content/exports/nwi_ccap_filled"
destination_dir = "/content/OneYear/utm4/labels"

# List all .tif files in the source directory
tif_files = [file for file in os.listdir(source_dir) if file.endswith(".tif")]

# Move each .tif file to the destination directory
for file in tif_files:
    # Define the source and destination file paths
    source_file = os.path.join(source_dir, file)
    destination_file = os.path.join(destination_dir, file)

    # Move the file to the destination directory
    shutil.move(source_file, destination_file)

In [29]:
# Define the folder containing .tif files
images_folder = '/content/OneYear/utm4/labels'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')


The number of .tif files in the folder is: 1466


# TwoYears UTM 4 Data

In [12]:
!unzip /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4_updatedExports_march20/utm4_TwoYears_stacked_01.zip -d /content/TwoYears/utm4/images/

Archive:  /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4_updatedExports_march20/utm4_TwoYears_stacked_01.zip
  inflating: /content/TwoYears/utm4/images/content/folder_stack/TwoYears/30_stacked.tif  
  inflating: /content/TwoYears/utm4/images/content/folder_stack/TwoYears/575_stacked.tif  
  inflating: /content/TwoYears/utm4/images/content/folder_stack/TwoYears/468_stacked.tif  
  inflating: /content/TwoYears/utm4/images/content/folder_stack/TwoYears/61_stacked.tif  
  inflating: /content/TwoYears/utm4/images/content/folder_stack/TwoYears/425_stacked.tif  
  inflating: /content/TwoYears/utm4/images/content/folder_stack/TwoYears/98_stacked.tif  
  inflating: /content/TwoYears/utm4/images/content/folder_stack/TwoYears/239_stacked.tif  
  inflating: /content/TwoYears/utm4/images/content/folder_stack/TwoYears/504_stacked.tif  
  inflating: /content/TwoYears/utm4/images/content/folder_stack/TwoYears/167_stacked.tif  
  inflating: /content/TwoYears/utm4/images

In [13]:
# Define the source and destination directories
source_dir = "/content/TwoYears/utm4/images/content/folder_stack/TwoYears"
destination_dir = "/content/TwoYears/utm4/images"

# List all .tif files in the source directory
tif_files = [file for file in os.listdir(source_dir) if file.endswith(".tif")]

# Move each .tif file to the destination directory
for file in tif_files:
    # Define the source and destination file paths
    source_file = os.path.join(source_dir, file)
    destination_file = os.path.join(destination_dir, file)

    # Move the file to the destination directory
    shutil.move(source_file, destination_file)

In [14]:
# Define the folder containing .tif files
images_folder = '/content/TwoYears/utm4/images'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 2605


# OneYear UTM 5 Data

In [15]:
!unzip /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5_updatedExports_march20/utm5_OneYear_stacked.zip -d /content/OneYear/utm5/images/

Archive:  /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5_updatedExports_march20/utm5_OneYear_stacked.zip
   creating: /content/OneYear/utm5/images/content/stack/OneYear/
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/30_stacked.tif  
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/61_stacked.tif  
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/98_stacked.tif  
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/167_stacked.tif  
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/62_stacked.tif  
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/65_stacked.tif  
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/118_stacked.tif  
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/59_stacked.tif  
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/148_stacked.tif  
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/2

In [16]:
# Define the source and destination directories
source_dir = "/content/OneYear/utm5/images/content/stack/OneYear"
destination_dir = "/content/OneYear/utm5/images"

# List all .tif files in the source directory
tif_files = [file for file in os.listdir(source_dir) if file.endswith(".tif")]

# Move each .tif file to the destination directory
for file in tif_files:
    # Define the source and destination file paths
    source_file = os.path.join(source_dir, file)
    destination_file = os.path.join(destination_dir, file)

    # Move the file to the destination directory
    shutil.move(source_file, destination_file)

In [17]:
# Define the folder containing .tif files
images_folder = '/content/OneYear/utm5/images'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 229


In [18]:
!unzip /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5_updatedExports_march20/utm5_TwoYears_stacked.zip -d /content/TwoYears/utm5/images/

Archive:  /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5_updatedExports_march20/utm5_TwoYears_stacked.zip
   creating: /content/TwoYears/utm5/images/content/stack/TwoYears/
  inflating: /content/TwoYears/utm5/images/content/stack/TwoYears/30_stacked.tif  
  inflating: /content/TwoYears/utm5/images/content/stack/TwoYears/61_stacked.tif  
  inflating: /content/TwoYears/utm5/images/content/stack/TwoYears/98_stacked.tif  
  inflating: /content/TwoYears/utm5/images/content/stack/TwoYears/167_stacked.tif  
  inflating: /content/TwoYears/utm5/images/content/stack/TwoYears/62_stacked.tif  
  inflating: /content/TwoYears/utm5/images/content/stack/TwoYears/65_stacked.tif  
  inflating: /content/TwoYears/utm5/images/content/stack/TwoYears/118_stacked.tif  
  inflating: /content/TwoYears/utm5/images/content/stack/TwoYears/59_stacked.tif  
  inflating: /content/TwoYears/utm5/images/content/stack/TwoYears/148_stacked.tif  
  inflating: /content/TwoYears/utm5/images/c

In [19]:
# Define the source and destination directories
source_dir = "/content/TwoYears/utm5/images/content/stack/TwoYears"
destination_dir = "/content/TwoYears/utm5/images"

# List all .tif files in the source directory
tif_files = [file for file in os.listdir(source_dir) if file.endswith(".tif")]

# Move each .tif file to the destination directory
for file in tif_files:
    # Define the source and destination file paths
    source_file = os.path.join(source_dir, file)
    destination_file = os.path.join(destination_dir, file)

    # Move the file to the destination directory
    shutil.move(source_file, destination_file)

In [20]:
# Define the folder containing .tif files
images_folder = '/content/TwoYears/utm5/images'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 229


# For UTM 4 data, find the unmatched files

In [ ]:
# Specify the folder path
folder_path = '/content/TwoYears/utm5/images'

# Function to extract the numeric part of the filename
def extract_numeric_part(filename):
    match = re.match(r"(\d+)", filename)
    return int(match.group()) if match else None

# List all files in the folder
files = os.listdir(folder_path)

# Extract numeric parts and sort
numeric_parts = [extract_numeric_part(file) for file in files if extract_numeric_part(file) is not None]
sorted_numeric_parts = sorted(numeric_parts)

# Find missing numbers
max_num = sorted_numeric_parts[-1]  # Assuming the largest number in the sequence is the last one
complete_range = set(range(1, max_num + 1))  # Start from 1 to max_num, inclusive
missing_numbers = complete_range - set(sorted_numeric_parts)

# Print missing numbers
print("Missing numbers:", sorted(list(missing_numbers)))

Missing numbers: []


In [ ]:
# from collections import Counter

# # Are there any duplicate files?
# # No duplicate files found.

# # Specify the folder path
# folder_path = '/content/TwoYears/utm5/images'

# # Function to extract the numeric part of the filename
# def extract_numeric_part(filename):
#     match = re.match(r"(\d+)", filename)
#     return int(match.group()) if match else None

# # List all files in the folder
# files = os.listdir(folder_path)

# # Extract numeric parts
# numeric_parts = [extract_numeric_part(file) for file in files if extract_numeric_part(file) is not None]

# # Count occurrences of each numeric part
# numeric_parts_counts = Counter(numeric_parts)

# # Identify and print duplicates
# duplicates = {part: count for part, count in numeric_parts_counts.items() if count > 1}
# if duplicates:
#     print("Duplicate files (numeric part: count):")
#     for part, count in duplicates.items():
#         print(f"{part}: {count} times")
# else:
#     print("No duplicate files found.")

No duplicate files found.


In [21]:
!mkdir /content/OneYear/utm4/REMOVED

In [23]:

# Define the source and destination directories
source_folder = "/content/OneYear/utm4/images/"
destination_folder = "/content/OneYear/utm4/REMOVED/"

# List of specific file names to move
files_to_move = ["596_stacked.tif", "597_stacked.tif", "598_stacked.tif", "599_stacked.tif", "600_stacked.tif",
                 "601_stacked.tif", "602_stacked.tif", "603_stacked.tif", "604_stacked.tif", "605_stacked.tif",
                 "606_stacked.tif", "607_stacked.tif", "608_stacked.tif", "609_stacked.tif", "610_stacked.tif",
                 "611_stacked.tif", "612_stacked.tif", "613_stacked.tif", "614_stacked.tif", "1832_stacked.tif",
                 "1833_stacked.tif", "1834_stacked.tif", "1835_stacked.tif", "1836_stacked.tif"]

# Move each file to the destination folder
for filename in files_to_move:
    # Construct the source and destination file paths
    source_file = os.path.join(source_folder, filename)
    destination_file = os.path.join(destination_folder, filename)

    # Check if the source file exists
    if os.path.exists(source_file):
        # Move the file to the destination folder
        shutil.move(source_file, destination_file)
        print(f"File '{filename}' moved successfully.")
    else:
        print(f"File '{filename}' does not exist in the source folder.")


File '596_stacked.tif' does not exist in the source folder.
File '597_stacked.tif' does not exist in the source folder.
File '598_stacked.tif' does not exist in the source folder.
File '599_stacked.tif' does not exist in the source folder.
File '600_stacked.tif' does not exist in the source folder.
File '601_stacked.tif' does not exist in the source folder.
File '602_stacked.tif' does not exist in the source folder.
File '603_stacked.tif' does not exist in the source folder.
File '604_stacked.tif' does not exist in the source folder.
File '605_stacked.tif' does not exist in the source folder.
File '606_stacked.tif' does not exist in the source folder.
File '607_stacked.tif' does not exist in the source folder.
File '608_stacked.tif' does not exist in the source folder.
File '609_stacked.tif' does not exist in the source folder.
File '610_stacked.tif' does not exist in the source folder.
File '611_stacked.tif' does not exist in the source folder.
File '612_stacked.tif' does not exist in

In [24]:
# Define the folder containing .tif files
images_folder = '/content/OneYear/utm4/images'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 2605


In [30]:
# Define the folder containing .tif files
images_folder = '/content/TwoYears/utm4/images'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 2605


# Rename files to include _utm4 or _utm5 for iamgery and save to google drive

In [39]:
def rename_files(folder_path):
    # List all .tif files in the folder
    files = [f for f in os.listdir(folder_path) if f.endswith('.tif')]

    for filename in files:
        # Check if the file is already in the desired format
        if "_planet_stack_utm4.tif" in filename:
            continue  # Skip renaming if it's already in the correct format

        # Construct the new filename by replacing '_stacked.tif' with '_planet_stack_utm4.tif'
        new_filename = filename.replace('_stacked.tif', '_planet_stack_utm4.tif')

        # Construct the full old and new file paths
        old_file_path = os.path.join(folder_path, filename)
        new_file_path = os.path.join(folder_path, new_filename)

        # Rename the file
        os.rename(old_file_path, new_file_path)
        print(f"Renamed '{filename}' to '{new_filename}'")

# Specify the path to your folder
folder_path = "/content/TwoYears/utm4/images"

# Call the function for the folder
rename_files(folder_path)

Renamed '1096_stacked.tif' to '1096_planet_stack_utm4.tif'
Renamed '2064_stacked.tif' to '2064_planet_stack_utm4.tif'
Renamed '343_stacked.tif' to '343_planet_stack_utm4.tif'
Renamed '244_stacked.tif' to '244_planet_stack_utm4.tif'
Renamed '2061_stacked.tif' to '2061_planet_stack_utm4.tif'
Renamed '431_stacked.tif' to '431_planet_stack_utm4.tif'
Renamed '325_stacked.tif' to '325_planet_stack_utm4.tif'
Renamed '2202_stacked.tif' to '2202_planet_stack_utm4.tif'
Renamed '563_stacked.tif' to '563_planet_stack_utm4.tif'
Renamed '1284_stacked.tif' to '1284_planet_stack_utm4.tif'
Renamed '678_stacked.tif' to '678_planet_stack_utm4.tif'
Renamed '2307_stacked.tif' to '2307_planet_stack_utm4.tif'
Renamed '1285_stacked.tif' to '1285_planet_stack_utm4.tif'
Renamed '2217_stacked.tif' to '2217_planet_stack_utm4.tif'
Renamed '1295_stacked.tif' to '1295_planet_stack_utm4.tif'
Renamed '2186_stacked.tif' to '2186_planet_stack_utm4.tif'
Renamed '2149_stacked.tif' to '2149_planet_stack_utm4.tif'
Renamed '

In [40]:
def copy_files_to_new_folder(source_folders, dest_folder):
    # Ensure the destination folder exists
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    for folder in source_folders:
        # List all files in the current source folder
        files = os.listdir(folder)

        for file in files:
            # Construct the path to the source file
            source_file = os.path.join(folder, file)

            # Check if it's a file (and not a directory)
            if os.path.isfile(source_file):
                # Construct the path to the destination file
                dest_file = os.path.join(dest_folder, file)

                # Copy the file to the destination folder
                shutil.copy(source_file, dest_file)
                print(f"Copied {file} to {dest_folder}")

# Specify the paths to your source folders
source_folders = [
    "/content/TwoYears/utm4/images"
]

# Specify the path to your new destination folder
dest_folder = "/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/TwoYear_stack_data"

# Copy files from both source folders to the destination folder
copy_files_to_new_folder(source_folders, dest_folder)

Copied 92_planet_stack_utm4.tif to /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/TwoYear_stack_data
Copied 334_planet_stack_utm4.tif to /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/TwoYear_stack_data
Copied 1324_planet_stack_utm4.tif to /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/TwoYear_stack_data
Copied 1776_planet_stack_utm4.tif to /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/TwoYear_stack_data
Copied 560_planet_stack_utm4.tif to /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/TwoYear_stack_data
Copied 1505_planet_stack_utm4.tif to /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/TwoYear_stack_data
Copied 2094_planet_stack_utm4.tif to /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/TwoYear_stack_data
Copied 1123_planet_stack_utm4.tif to /content/drive/

In [36]:
# Define the folder containing .tif files
images_folder = '/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/TwoYear_stack_data'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 229


# investigate the files across one, two, three year stack for utm 4 and utm 5.

In [50]:
# Do the number of files in the folders dont match. Look at the data across one, two, three stack folders (UTM5)

def find_matching_files(images_folder, labels_folder):
    # List all .tif files in both folders
    images_files = [f for f in os.listdir(images_folder) if f.endswith('.tif')]
    labels_files = [f for f in os.listdir(labels_folder) if f.endswith('.tif')]

    # Extract the common identifiers from the filenames
    images_ids = set(sorted([f.split('_planet_stack_utm4.tif')[0] for f in images_files]))
    labels_ids = set(sorted([f.split('_planet_stack_utm4.tif')[0] for f in labels_files]))

    # Find the intersection of identifiers, i.e., those that exist in both folders
    matching_ids = images_ids.intersection(labels_ids)

    # Construct the filenames based on matching identifiers
    matching_images = [f"{id}_planet_stack_utm4.tif" for id in matching_ids]
    matching_labels = [f"{id}_planet_stack_utm4.tif" for id in matching_ids]

    return matching_images, matching_labels

# Specify the paths to your folders
images_folder = "/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/TwoYear_stack_data"
labels_folder = "/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/ThreeYear_stack_data"

# Find matching files
matching_images, matching_labels = find_matching_files(images_folder, labels_folder)

# Print the count of matching files
print(f"Found {len(matching_images)} matching image(s) and {len(matching_labels)} matching image(s).")

# Example output: matching filenames
for img, lbl in zip(matching_images, matching_labels):
    print(f"Image: {img} <--> Image: {lbl}")

Found 2605 matching image(s) and 2605 matching image(s).
Image: 1044_planet_stack_utm4.tif <--> Image: 1044_planet_stack_utm4.tif
Image: 2639_planet_stack_utm4.tif <--> Image: 2639_planet_stack_utm4.tif
Image: 753_planet_stack_utm4.tif <--> Image: 753_planet_stack_utm4.tif
Image: 1828_planet_stack_utm4.tif <--> Image: 1828_planet_stack_utm4.tif
Image: 1199_planet_stack_utm4.tif <--> Image: 1199_planet_stack_utm4.tif
Image: 1329_planet_stack_utm4.tif <--> Image: 1329_planet_stack_utm4.tif
Image: 1880_planet_stack_utm4.tif <--> Image: 1880_planet_stack_utm4.tif
Image: 211_planet_stack_utm4.tif <--> Image: 211_planet_stack_utm4.tif
Image: 1187_planet_stack_utm4.tif <--> Image: 1187_planet_stack_utm4.tif
Image: 1995_planet_stack_utm4.tif <--> Image: 1995_planet_stack_utm4.tif
Image: 2430_planet_stack_utm4.tif <--> Image: 2430_planet_stack_utm4.tif
Image: 824_planet_stack_utm4.tif <--> Image: 824_planet_stack_utm4.tif
Image: 1082_planet_stack_utm4.tif <--> Image: 1082_planet_stack_utm4.tif


In [70]:
import os

# Define the folder paths
folder1_path = "/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/OneYear_stack_data"
folder2_path = "/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/ThreeYear_stack_data"
folder3_path = "/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/nwi_ccap_filled"

# # Function to get sorted list of files in a folder
# def get_sorted_files(folder_path):
#     files = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
#     sorted_files = sorted(files, key=lambda x: int(x.split('.')[0]))
#     return sorted_files




# Function to get sorted list of files in a folder
def get_sorted_files(folder_path):
    files = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
    sorted_files = sorted(files, key=lambda x: int(x.split('_')[0]) if x.split('_')[0].isdigit() else x)
    return sorted_files


# Get sorted lists of files in all three folders
folder1_files = get_sorted_files(folder1_path)
folder2_files = get_sorted_files(folder2_path)
folder3_files = get_sorted_files(folder3_path)

# Check if the lists match across all three folders
if folder1_files == folder2_files == folder3_files:
    print("The lists of files in all three folders match.")
else:
    print("The lists of files in all three folders do not match.")

# # Print files in each folder
# print("Files in Folder 1:")
# for file in folder1_files:
#     print(file)

# print("\nFiles in Folder 2:")
# for file in folder2_files:
#     print(file)

print("\nFiles in Folder 3:")
for file in folder3_files:
    print(file)


The lists of files in all three folders do not match.

Files in Folder 3:
1_nwi_ccap_filled_utm4.tif
2_nwi_ccap_filled_utm4.tif
3_nwi_ccap_filled_utm4.tif
4_nwi_ccap_filled_utm4.tif
5_nwi_ccap_filled_utm4.tif
6_nwi_ccap_filled_utm4.tif
7_nwi_ccap_filled_utm4.tif
8_nwi_ccap_filled_utm4.tif
9_nwi_ccap_filled_utm4.tif
10_nwi_ccap_filled_utm4.tif
11_nwi_ccap_filled_utm4.tif
12_nwi_ccap_filled_utm4.tif
13_nwi_ccap_filled_utm4.tif
14_nwi_ccap_filled_utm4.tif
15_nwi_ccap_filled_utm4.tif
16_nwi_ccap_filled_utm4.tif
17_nwi_ccap_filled_utm4.tif
18_nwi_ccap_filled_utm4.tif
19_nwi_ccap_filled_utm4.tif
20_nwi_ccap_filled_utm4.tif
21_nwi_ccap_filled_utm4.tif
22_nwi_ccap_filled_utm4.tif
23_nwi_ccap_filled_utm4.tif
24_nwi_ccap_filled_utm4.tif
25_nwi_ccap_filled_utm4.tif
26_nwi_ccap_filled_utm4.tif
27_nwi_ccap_filled_utm4.tif
28_nwi_ccap_filled_utm4.tif
29_nwi_ccap_filled_utm4.tif
30_nwi_ccap_filled_utm4.tif
31_nwi_ccap_filled_utm4.tif
32_nwi_ccap_filled_utm4.tif
33_nwi_ccap_filled_utm4.tif
34_nwi_ccap

In [94]:
# !ls {folder1_path}
# !ls {folder2_path}
!ls {"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/nwi_ccap_filled"}

100_nwi_ccap_filled_utm5.tif  170_nwi_ccap_filled_utm5.tif  32_nwi_ccap_filled_utm5.tif
101_nwi_ccap_filled_utm5.tif  171_nwi_ccap_filled_utm5.tif  33_nwi_ccap_filled_utm5.tif
102_nwi_ccap_filled_utm5.tif  172_nwi_ccap_filled_utm5.tif  34_nwi_ccap_filled_utm5.tif
103_nwi_ccap_filled_utm5.tif  173_nwi_ccap_filled_utm5.tif  35_nwi_ccap_filled_utm5.tif
104_nwi_ccap_filled_utm5.tif  174_nwi_ccap_filled_utm5.tif  36_nwi_ccap_filled_utm5.tif
105_nwi_ccap_filled_utm5.tif  175_nwi_ccap_filled_utm5.tif  37_nwi_ccap_filled_utm5.tif
106_nwi_ccap_filled_utm5.tif  176_nwi_ccap_filled_utm5.tif  38_nwi_ccap_filled_utm5.tif
107_nwi_ccap_filled_utm5.tif  177_nwi_ccap_filled_utm5.tif  39_nwi_ccap_filled_utm5.tif
108_nwi_ccap_filled_utm5.tif  178_nwi_ccap_filled_utm5.tif  3_nwi_ccap_filled_utm5.tif
109_nwi_ccap_filled_utm5.tif  179_nwi_ccap_filled_utm5.tif  40_nwi_ccap_filled_utm5.tif
10_nwi_ccap_filled_utm5.tif   17_nwi_ccap_filled_utm5.tif   41_nwi_ccap_filled_utm5.tif
110_nwi_ccap_filled_utm5.tif  180

In [73]:
# Define the folder containing .tif files
images_folder = '/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 2825


# Time to MOVE utm4 and utm5 oneyear, twoyear, threeyear data to folders for modeling

In [99]:
tif_source  = "/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/nwi_ccap_filled/"
export_path = "/content/drive/MyDrive/DeepLearning_Project/planet_TwoYear/labels/"

# Ensure the destination folder exists
if not os.path.exists(export_path):
    os.makedirs(export_path)

# List all .tif files in the source directory
tif_files = [f for f in os.listdir(tif_source) if f.endswith('.tif')]

# Copy each .tif file to the Google Drive destination folder
for tif_file in tif_files:
    source_path = os.path.join(tif_source, tif_file)
    destination_path = os.path.join(export_path, tif_file)
    shutil.copy(source_path, destination_path)
    print(f"Copied {tif_file} to Google Drive folder.")

Copied 23_nwi_ccap_filled_utm5.tif to Google Drive folder.
Copied 184_nwi_ccap_filled_utm5.tif to Google Drive folder.
Copied 20_nwi_ccap_filled_utm5.tif to Google Drive folder.
Copied 58_nwi_ccap_filled_utm5.tif to Google Drive folder.
Copied 116_nwi_ccap_filled_utm5.tif to Google Drive folder.
Copied 1_nwi_ccap_filled_utm5.tif to Google Drive folder.
Copied 38_nwi_ccap_filled_utm5.tif to Google Drive folder.
Copied 189_nwi_ccap_filled_utm5.tif to Google Drive folder.
Copied 181_nwi_ccap_filled_utm5.tif to Google Drive folder.
Copied 137_nwi_ccap_filled_utm5.tif to Google Drive folder.
Copied 147_nwi_ccap_filled_utm5.tif to Google Drive folder.
Copied 101_nwi_ccap_filled_utm5.tif to Google Drive folder.
Copied 122_nwi_ccap_filled_utm5.tif to Google Drive folder.
Copied 67_nwi_ccap_filled_utm5.tif to Google Drive folder.
Copied 168_nwi_ccap_filled_utm5.tif to Google Drive folder.
Copied 64_nwi_ccap_filled_utm5.tif to Google Drive folder.
Copied 80_nwi_ccap_filled_utm5.tif to Google Dri

In [100]:
# Define the folder containing .tif files
images_folder = '/content/drive/MyDrive/DeepLearning_Project/planet_TwoYear/labels'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 2834
